In [0]:
# import the required libraries
from tensorflow import keras
from tensorflow.keras import backend as K
import numpy as np
from tensorflow.keras.datasets import cifar10, mnist
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from time import time
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf

from tensorflow.python.client import device_lib

In [0]:
tf.test.gpu_device_name()
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 17330517982695299895, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13638376892604354678
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 5431002843763433876
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14800692839
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11708527400363893349
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [0]:
#GPU count and name
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-28ee3479-2ee7-91f7-835c-3c6dec45dc39)


In [0]:
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Tue May  7 05:52:29 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    30W /  70W |    611MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# information data
batch_size = 64
num_classes = 10
epochs = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print('x_train shape :',x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape : (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [0]:
#Define the model with Standard Network Models
visible = Input(shape=(28,28,1))
conv2d_1 = Conv2D(16, kernel_size=(5, 5), strides=1, input_shape=input_shape, padding='same')(visible)
pooling_1 = MaxPooling2D(pool_size=(2, 2))(conv2d_1)
conv2d_2 = Conv2D(36, kernel_size=(5, 5), strides=1, input_shape=input_shape, padding='same')(pooling_1)
pooling_2 = MaxPooling2D(pool_size=(2, 2))(conv2d_2)
flatten_1 = Flatten()(pooling_2)
dense_1 = Dense(128)(flatten_1)
dropout= Dropout(0.2)(dense_1)
output = Dense(num_classes, activation='softmax')(dropout)

model = Model(inputs=visible, outputs=output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 28, 28, 16)        416       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 36)        14436     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 7, 7, 36)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 1764)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               225920    
__________

In [0]:
from tensorflow.keras import backend as K
# losses in keras
from tensorflow.keras.losses import (   mean_squared_error,
                                        mean_absolute_error,
                                        mean_absolute_percentage_error,
                                        mean_squared_logarithmic_error,
                                        squared_hinge,
                                        hinge,
                                        categorical_hinge,
                                        logcosh,
                                        categorical_crossentropy,
                                        sparse_categorical_crossentropy,
                                        binary_crossentropy,
                                        kullback_leibler_divergence,
                                        poisson,
                                        cosine_proximity)

# define my loss
def my_losses_mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

In [0]:
# define optimizers
from tensorflow.keras.optimizers import ( SGD, 
                                         RMSprop, Adagrad, Adadelta, 
                                         Adam, Adamax, Nadam)
# Compile the model
model.compile(loss=categorical_crossentropy,
             optimizer=Adam(),
             metrics=['accuracy'])

In [0]:
#Train the model using tensorboard instance in the callbacks
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 81us/sample - loss: 0.1585 - acc: 0.9522 - val_loss: 0.0744 - val_acc: 0.9761
Epoch 2/10
60000/60000 [==============================] - 5s 76us/sample - loss: 0.0679 - acc: 0.9790 - val_loss: 0.0557 - val_acc: 0.9815
Epoch 3/10
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0533 - acc: 0.9825 - val_loss: 0.0544 - val_acc: 0.9819
Epoch 4/10
60000/60000 [==============================] - 5s 75us/sample - loss: 0.0456 - acc: 0.9854 - val_loss: 0.0477 - val_acc: 0.9846
Epoch 5/10
60000/60000 [==============================] - 5s 76us/sample - loss: 0.0395 - acc: 0.9876 - val_loss: 0.0508 - val_acc: 0.9840
Epoch 6/10
60000/60000 [==============================] - 5s 75us/sample - loss: 0.0363 - acc: 0.9883 - val_loss: 0.0460 - val_acc: 0.9864
Epoch 7/10
60000/60000 [==============================] - 4s 75us/sample - loss: 0.0329 - acc: 0.9896 - val_loss: 0.

In [0]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.058978839240674645
Test accuracy: 0.9852
